In [1]:
import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader 
import sys 
sys.path.append('../')

from data_yours import ALS_50K

In [2]:
# get sample 
ds = ALS_50K(num_points=2048)
sample = ds[25542]

In [3]:
# get xyz 
sample = sample[0]
sample.shape

(2048, 3)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
sample = torch.tensor(sample, dtype=torch.float32).to(device)  # add batch dimension

## For a single point cloud

## kNN Clustering function 

In [6]:
from kmeans_pytorch import kmeans

In [ ]:
def get_tree_center_xy(pc): 
    return torch.mean(pc[:,:2], dim=0)
get_tree_center_xy(sample)

In [36]:
def initialize_clusters(pc, device, k=100):
    c_idx, c_centroids = kmeans(
        X=pc,
        num_clusters=k,
        distance='euclidean',
        device=device
    )
    return c_idx, c_centroids
cl_idx, cl_centroids = initialize_clusters(sample, device, k=100)
cl_idx.shape, cl_centroids.shape

running k-means on cuda..


[running kmeans]: 11it [00:00, 57.28it/s, center_shift=0.000000, iteration=11, tol=0.000100]


(torch.Size([2048]), torch.Size([100, 3]))

In [37]:
def get_clusters(pc, c_idx):
    """
    Splits the point cloud (pc) into clusters according to c_idx.
    Returns a list of tensors, one per cluster.
    """
    return [pc[c_idx == i] for i in range(c_idx.max() + 1)]
clusters = get_clusters(sample, cl_idx)
len(clusters), clusters[0].shape

(100, torch.Size([16, 3]))

In [38]:
def compute_pca(points):
    mean = points.mean(dim=0)
    X = points - mean
    eigvals, eigvecs = torch.linalg.eigh(torch.cov(X.T))
    idx = torch.argsort(eigvals, descending=True)
    eigvecs = eigvecs[:, idx]
    direction = eigvecs[:, 0]  # 1st principal component
    normal = eigvecs[:, 1]     # 2nd principal component
    return mean, direction, normal
compute_pca(clusters[0])

(tensor([ 0.0554,  0.1117, -0.2008], device='cuda:0'),
 tensor([-0.2600, -0.0675,  0.9633], device='cuda:0'),
 tensor([-0.5133, -0.8353, -0.1970], device='cuda:0'))

In [39]:
def cluster_residuals(cluster):
    centroid, direction, normal = compute_pca(cluster)
    vec = cluster - centroid 
    normal = normal / torch.norm(normal) 
    distances = torch.abs(torch.matmul(vec, normal))
    return torch.sum(distances)
cluster_residuals(clusters[0])


tensor(0.5534, device='cuda:0')

In [46]:
def merged_residual_diff_percent(c1, c2):
    r_prior = cluster_residuals(c1) + cluster_residuals(c2)
    r_post = cluster_residuals(merge_clusters(c1, c2))
    return (r_post - r_prior) / r_prior * 100.0 

merged_residual_diff_percent(c1, c2)

tensor(70.6257, device='cuda:0')

In [41]:
def distance_pairs(centroids): 
    dmatrix = torch.cdist(centroids, centroids) 
    dmatrix = dmatrix.fill_diagonal_(float('inf'))
    tril_indices = torch.tril_indices(dmatrix.size(0), dmatrix.size(1), offset=-1)
    distances = dmatrix[tril_indices[0], tril_indices[1]]
    sorted_indices = distances.argsort()
    sorted_pairs = torch.stack((tril_indices[0][sorted_indices], tril_indices[1][sorted_indices]), dim=1)
    sorted_distances = distances[sorted_indices]
    return sorted_pairs, sorted_distances

distance_pairs(cl_centroids)

(tensor([[89, 67],
         [32, 30],
         [79, 15],
         ...,
         [60, 23],
         [97, 23],
         [40, 23]]),
 tensor([0.0591, 0.0627, 0.0654,  ..., 1.7361, 1.7436, 1.7535]))

In [42]:
def pairs_angle(c1, c2): 
    c1_mean, c1_direction, c1_normal = compute_pca(c1)
    c2_mean, c2_direction, c2_normal = compute_pca(c2)
    angle = torch.acos(torch.dot(c1_normal, c2_normal) / (torch.norm(c1_normal) * torch.norm(c2_normal)))
    return angle

c1 = clusters[89]
c2 = clusters[67]
pairs_angle(c1, c2)

tensor(0.6252, device='cuda:0')

In [35]:
def merge_clusters(c1, c2):
    merged = torch.cat((c1, c2), dim=0)
    return merged

print(c1.shape, c2.shape)
merged = merge_clusters(c1, c2)
print(merged.shape)

torch.Size([8, 3]) torch.Size([6, 3])
torch.Size([14, 3])


In [49]:
def compute_threshold(rdpercents): 
    return torch.quantile(rdpercents, 0.75)

In [58]:
# all the way stage one test 
c_idx, c_centroids = initialize_clusters(sample, device, k=100)
clusters = get_clusters(sample, cl_idx)
pairs, distances = distance_pairs(c_centroids)

rdpercent = [] 
for i, (c1_idx, c2_idx) in enumerate(pairs):
    c1 = clusters[c1_idx]
    c2 = clusters[c2_idx]
    rdpercent.append(merged_residual_diff_percent(c1, c2))
rpercent = torch.tensor(rdpercent, dtype=torch.float32).to(device)

running k-means on cuda..


[running kmeans]: 0it [00:00, ?it/s, center_shift=12.905282, iteration=1, tol=0.000100]

[running kmeans]: 13it [00:00, 51.74it/s, center_shift=0.000000, iteration=13, tol=0.000100]


In [60]:
t = compute_threshold(rpercent)
t

tensor(69.5705, device='cuda:0')

In [72]:
def sort_pairs_distance_angle(pairs, distances, angles):
    # Sort by distance + small weight on angle
    sort_metric = distances + 1e-2 * angles
    sorted_indices = torch.argsort(sort_metric)
    sorted_pairs = pairs[sorted_indices]
    return sorted_pairs

# get all angles per pair 
angles = torch.tensor([pairs_angle(clusters[pair[0]], clusters[pair[1]]) for pair in pairs], dtype=torch.float32).to(device)
# Ensure distances is on the same device as angles
distances = distances.to(device)
sorted_pairs = sort_pairs_distance_angle(pairs, distances, angles)

In [73]:
sorted_pairs

tensor([[59, 21],
        [35, 33],
        [50, 21],
        ...,
        [27,  8],
        [27,  7],
        [49, 27]], device='cuda:0')

In [78]:
diff_percen = [merged_residual_diff_percent(clusters[pair[0]], clusters[pair[1]]) for pair in sorted_pairs]

In [81]:
diff_percent = torch.tensor(diff_percen, dtype=torch.float32).to(device)

In [88]:
below_threshold_indices = torch.nonzero(diff_percent < t, as_tuple=False).squeeze()

In [108]:
# all the way stage one test 
c_idx, c_centroids = initialize_clusters(sample, device, k=100)
clusters = get_clusters(sample, cl_idx)

running k-means on cuda..


[running kmeans]: 13it [00:00, 50.35it/s, center_shift=0.000000, iteration=13, tol=0.000100]


In [ ]:
c_idx, c_centroids = initialize_clusters(sample, device, k=100)

clusters = get_clusters(sample, cl_idx)
pairs, distances = distance_pairs(c_centroids)
distances = distances.to(device)
angles = torch.tensor([pairs_angle(clusters[pair[0]], clusters[pair[1]]) for pair in pairs], dtype=torch.float32).to(device)
sorted_pairs = sort_pairs_distance_angle(pairs.to(device), distances, angles)


running k-means on cuda..


[running kmeans]: 0it [00:00, ?it/s, center_shift=10.314820, iteration=1, tol=0.000100]

[running kmeans]: 13it [00:00, 46.49it/s, center_shift=0.000068, iteration=13, tol=0.000100]


In [119]:
# get threshold 
rdpercent = []
for i, (c1_idx, c2_idx) in enumerate(sorted_pairs):
    c1 = clusters[c1_idx]
    c2 = clusters[c2_idx]
    rdpercent.append(merged_residual_diff_percent(c1, c2))
rpercent = torch.tensor(rdpercent, dtype=torch.float32).to(device)
t = compute_threshold(rpercent)